In [12]:
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np

import pycuda
import cupy as cp

# from pyculib.blas import Blas
# Blas = Blas()

In [13]:

a_gpu = cp.random.randn(4, 4)
b_gpu = cp.random.randn(4, 4)

c_gpu = cp.dot(a_gpu, b_gpu)

In [18]:
a_gpu, b_gpu, c_gpu

(array([[-0.45208806, -1.45588358, -0.10702163, -1.93156534],
        [-2.57512826,  2.32087814, -0.81009215, -1.59609201],
        [-0.62343255, -1.1545197 ,  1.22625423, -0.23463489],
        [ 1.0176159 ,  0.19221372,  1.37841437, -1.05242602]]),
 array([[-0.74161874, -0.46466223, -0.24858672, -0.58552531],
        [-0.33044028, -0.12392781,  1.26150275,  1.19077757],
        [ 1.06636659,  2.06785923,  2.2291372 ,  1.65695417],
        [-0.70977577,  0.77419017,  0.74723562, -0.03498322]]),
 array([[ 2.07321354, -1.32621186, -3.4061184 , -1.57868207],
        [ 1.41186398, -2.00189179,  0.56947349,  2.98500328],
        [ 2.31802379,  2.78683177,  1.25670865,  1.03031472],
        [ 1.39868333,  1.5389207 ,  2.27577688,  1.95383061]]))

In [9]:
a_gpu = gpuarray.to_gpu(numpy.random.randn(4, 4).astype(numpy.float32))
b_gpu = gpuarray.to_gpu(numpy.random.randn(4, 4).astype(numpy.float32))
c_gpu = gpuarray.to_gpu(numpy.zeros((4,4)).astype(numpy.float32))


# Blas.gemm('N', 'N', 4, 4, 4, a_gpu, b_gpu, 0, c_gpu)

AttributeError: 'GPUArray' object has no attribute 'dot'

In [38]:
#!python 
# Sample source code from the Tutorial Introduction in the documentation.

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy
a = numpy.random.randn(4,4)

a = a.astype(numpy.float32)

a_gpu = cuda.mem_alloc(a.size * a.dtype.itemsize)

cuda.memcpy_htod(a_gpu, a)

mod = SourceModule("""
    __global__ void doublify(float *a)
    {
      int idx = threadIdx.x + threadIdx.y*4;
      a[idx] *= 2;
    }
    """)

func = mod.get_function("doublify")
func(a_gpu, block=(4, 4, 1))

a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print ("original array:")
print (a)
print ("doubled with gpuarray:")
print (a_doubled)

# alternate kernel invocation -------------------------------------------------

# func(cuda.InOut(a), block=(2, 2, 1))
# print ("doubled with InOut:")
# print (a)

# # part 2 ----------------------------------------------------------------------

# import pycuda.gpuarray as gpuarray
# a_gpu = gpuarray.to_gpu(numpy.random.randn(4,4).astype(numpy.float32))
# a_doubled = (2*a_gpu).get()

# print ("original array:")
# print (a_gpu)
# print ("doubled with gpuarray:")
# print (a_doubled)

original array:
[[-0.11195923 -0.22853392 -0.6305419  -0.19271427]
 [ 0.8398443   0.16593747  0.31804982  2.5873938 ]
 [-0.7881569   1.2949262  -0.79106194  1.0940986 ]
 [-1.3863486  -0.19165555 -1.1574366  -0.7090791 ]]
doubled with gpuarray:
[[-0.22391845 -0.45706785 -1.2610838  -0.38542855]
 [ 1.6796886   0.33187494  0.63609964  5.1747875 ]
 [-1.5763139   2.5898523  -1.5821239   2.1881971 ]
 [-2.7726972  -0.3833111  -2.3148732  -1.4181582 ]]


In [20]:
A = gpuarray.to_gpu(numpy.random.randn(4, 4).astype(numpy.float32))
B = gpuarray.to_gpu(numpy.random.randn(4, 4).astype(numpy.float32))
C = gpuarray.to_gpu(numpy.zeros((4, 4), dtype=numpy.float32))#.astype(numpy.float32))

# C = gpuarray.to_gpu(numpy.zeros(4, 4).astype(numpy.float32))
